Libraries and modules

In [351]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler

Reading the dataset

In [352]:
df = pd.read_csv("train_u6lujuX_CVtuZ9i.csv")
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


Handling of missing values and outliers

In [353]:
df["Loan_ID"].isnull().sum()

0

In [354]:
df["Loan_ID"].count()

614

In [355]:
for i in df.columns:
    print(f"Percentage of missing values in the column {i}: {df[i].isnull().sum()*100/614}")

Percentage of missing values in the column Loan_ID: 0.0
Percentage of missing values in the column Gender: 2.1172638436482085
Percentage of missing values in the column Married: 0.48859934853420195
Percentage of missing values in the column Dependents: 2.44299674267101
Percentage of missing values in the column Education: 0.0
Percentage of missing values in the column Self_Employed: 5.2117263843648205
Percentage of missing values in the column ApplicantIncome: 0.0
Percentage of missing values in the column CoapplicantIncome: 0.0
Percentage of missing values in the column LoanAmount: 3.583061889250814
Percentage of missing values in the column Loan_Amount_Term: 2.2801302931596092
Percentage of missing values in the column Credit_History: 8.143322475570033
Percentage of missing values in the column Property_Area: 0.0
Percentage of missing values in the column Loan_Status: 0.0


In [356]:
df = df.drop(["Loan_ID"], axis = 1)

In [357]:
for i in ["Gender", "Married", "Dependents", "Self_Employed", "Loan_Amount_Term", "Credit_History"]:
    df[i].fillna(df[i].mode()[0], inplace = True)

In [358]:
df["LoanAmount"].fillna(df["LoanAmount"].mean(), inplace = True)

In [360]:
df = pd.get_dummies(df)

In [361]:
df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

In [362]:
df

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y
0,5849,0.0,146.412162,360.0,1.0,0,1,1,0,1,...,0,1,0,1,0,0,0,1,0,1
1,4583,1508.0,128.000000,360.0,1.0,0,1,0,1,0,...,0,1,0,1,0,1,0,0,1,0
2,3000,0.0,66.000000,360.0,1.0,0,1,0,1,1,...,0,1,0,0,1,0,0,1,0,1
3,2583,2358.0,120.000000,360.0,1.0,0,1,0,1,1,...,0,0,1,1,0,0,0,1,0,1
4,6000,0.0,141.000000,360.0,1.0,0,1,1,0,1,...,0,1,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,3232,1950.0,108.000000,360.0,1.0,0,1,0,1,1,...,0,1,0,1,0,1,0,0,0,1
609,2900,0.0,71.000000,360.0,1.0,1,0,1,0,1,...,0,1,0,1,0,1,0,0,0,1
611,8072,240.0,253.000000,360.0,1.0,0,1,0,1,0,...,0,1,0,1,0,0,0,1,0,1
612,7583,0.0,187.000000,360.0,1.0,0,1,0,1,0,...,0,1,0,1,0,0,0,1,0,1


In [370]:
q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
iqr = q3 - q1

print(~((df<(q1-1.5*iqr)) | (df>(q3+1.5*iqr))))

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0               True               True        True              True   
1               True               True        True              True   
2               True               True        True              True   
3               True               True        True              True   
4               True               True        True              True   
..               ...                ...         ...               ...   
127             True               True        True              True   
128             True               True        True              True   
129             True               True        True              True   
130             True               True        True              True   
131             True               True        True              True   

     Credit_History  Gender_Female  Gender_Male  Married_No  Married_Yes  \
0              True           True         True

In [364]:
q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
iqr = q3 - q1

df = df[~((df<(q1-1.5*iqr)) | (df>(q3+1.5*iqr))).any(axis = 1)]

In [365]:
df

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y
0,5849,0.0,146.412162,360.0,1.0,0,1,1,0,1,...,0,1,0,1,0,0,0,1,0,1
4,6000,0.0,141.000000,360.0,1.0,0,1,1,0,1,...,0,1,0,1,0,0,0,1,0,1
13,1853,2840.0,114.000000,360.0,1.0,0,1,1,0,1,...,0,1,0,1,0,1,0,0,1,0
15,4950,0.0,125.000000,360.0,1.0,0,1,1,0,1,...,0,1,0,1,0,0,0,1,0,1
19,2600,3500.0,115.000000,360.0,1.0,0,1,0,1,1,...,0,1,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,3182,2917.0,161.000000,360.0,1.0,0,1,1,0,1,...,0,1,0,1,0,0,0,1,0,1
586,2297,1522.0,104.000000,360.0,1.0,0,1,0,1,1,...,0,1,0,1,0,0,0,1,0,1
588,4750,0.0,94.000000,360.0,1.0,0,1,1,0,1,...,0,1,0,1,0,0,1,0,0,1
603,3676,4301.0,172.000000,360.0,1.0,0,1,1,0,1,...,0,1,0,1,0,1,0,0,0,1


In [366]:
df.reset_index()

,index,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,...,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y
0,0,5849,0.0,146.412162,360.0,1.0,0,1,1,0,...,0,1,0,1,0,0,0,1,0,1
1,4,6000,0.0,141.000000,360.0,1.0,0,1,1,0,...,0,1,0,1,0,0,0,1,0,1
2,13,1853,2840.0,114.000000,360.0,1.0,0,1,1,0,...,0,1,0,1,0,1,0,0,1,0
3,15,4950,0.0,125.000000,360.0,1.0,0,1,1,0,...,0,1,0,1,0,0,0,1,0,1
4,19,2600,3500.0,115.000000,360.0,1.0,0,1,0,1,...,0,1,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,579,3182,2917.0,161.000000,360.0,1.0,0,1,1,0,...,0,1,0,1,0,0,0,1,0,1
128,586,2297,1522.0,104.000000,360.0,1.0,0,1,0,1,...,0,1,0,1,0,0,0,1,0,1
129,588,4750,0.0,94.000000,360.0,1.0,0,1,1,0,...,0,1,0,1,0,0,1,0,0,1
130,603,3676,4301.0,172.000000,360.0,1.0,0,1,1,0,...,0,1,0,1,0,1,0,0,0,1


Handling of skewness

In [367]:
scaler = MinMaxScaler()

df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)